In [24]:
import pandas as pd
import numpy as np

# Load Fairface train and validation datasets
fairface_train = pd.read_csv('C:/Users/Shweta/Downloads/fairface_label_train.csv')
fairface_val = pd.read_csv('C:/Users/Shweta/Downloads/fairface_label_val.csv')

# Combine train and validation datasets
fairface = pd.concat([fairface_train, fairface_val], ignore_index=True)

# Convert 'age' column to numeric, forcing errors to NaN, then filter valid ages
fairface['age'] = pd.to_numeric(fairface['age'], errors='coerce')
fairface = fairface[(fairface['age'] >= 10) & (fairface['age'] <= 60)].dropna(subset=['age'])

# Keep only relevant columns
fairface = fairface[['file', 'age', 'gender', 'race', 'service_test']]


In [40]:
# Load and preprocess FER2013 dataset
fer2013 = pd.read_csv('C:/Users/Shweta/Downloads/fer2013.csv')

# Check if 'usage' is in columns
if 'usage' in fer2013.columns:
    fer2013 = fer2013[fer2013['usage'] == 'Training']
fer2013.rename(columns={'pixels': 'file'}, inplace=True)

# Convert 'file' to string for consistency
fer2013['file'] = fer2013['file'].astype(str)

# Keep only relevant columns
fer2013 = fer2013[['file', 'emotion']]


In [50]:
# Load Fashion dataset (example)
fashion = pd.read_csv('C:/Users/Shweta/Downloads/dress.csv')

# Keep only relevant columns and rename them for consistency
fashion = fashion[['image_url', 'category']]
fashion.rename(columns={'image_url': 'file', 'category': 'dress_color'}, inplace=True)




In [51]:
# Merge Fairface and FER2013 datasets on 'file'
merged_data = pd.merge(fairface, fer2013, on='file', how='inner')

# Merge the result with Fashion dataset on 'file'
merged_data = pd.merge(merged_data, fashion, on='file', how='inner')


In [52]:
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import numpy as np

def preprocess_image(image_path):
    image = load_img(image_path, target_size=(224, 224))
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    image /= 255.0
    return image


In [53]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Flatten, Dense, Dropout

# Input layer
input_image = Input(shape=(224, 224, 3))

# Convolutional layers
conv1 = Conv2D(32, kernel_size=(3, 3), activation='relu')(input_image)
conv2 = Conv2D(64, kernel_size=(3, 3), activation='relu')(conv1)
flatten = Flatten()(conv2)

# Fully connected layers for each task
fc1 = Dense(128, activation='relu')(flatten)
dropout1 = Dropout(0.5)(fc1)

# Output layers for each task
nationality_output = Dense(5, activation='softmax', name='nationality')(dropout1)
age_output = Dense(1, activation='linear', name='age')(dropout1)
dress_color_output = Dense(10, activation='softmax', name='dress_color')(dropout1)
emotion_output = Dense(7, activation='softmax', name='emotion')(dropout1)

# Model
model = Model(inputs=input_image, outputs=[nationality_output, age_output, dress_color_output, emotion_output])

# Compile the model
model.compile(optimizer='adam', 
              loss={'nationality': 'categorical_crossentropy', 
                    'age': 'mse', 
                    'dress_color': 'categorical_crossentropy', 
                    'emotion': 'categorical_crossentropy'},
              metrics={'nationality': 'accuracy', 
                       'age': 'mae', 
                       'dress_color': 'accuracy', 
                       'emotion': 'accuracy'})

# Print the model summary
model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 222, 222,  │        896 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 220, 220,  │     18,496 │ conv2d[0][0]      │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 3097600)   │          0 │ conv2d_1[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 128)       │ 396,492,9… │ flatten[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 128)       │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ nationality (Dense) │ (None, 5)         │        645 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ age (Dense)         │ (None, 1)         │        129 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dress_color (Dense) │ (None, 10)        │      1,290 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ emotion (Dense)     │ (None, 7)         │        903 │ dropout[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 396,515,287 (1.48 GB)

 Trainable params: 396,515,287 (1.48 GB)

 Non-trainable params: 0 (0.00 B)

In [54]:
# Split data into training and validation sets
from sklearn.model_selection import train_test_split

train_data, val_data = train_test_split(merged_data, test_size=0.2, random_state=42)

# Load images and preprocess them
X_train = np.array([preprocess_image(file) for file in train_data['file']])
X_val = np.array([preprocess_image(file) for file in val_data['file']])

# Prepare labels
y_train_nationality = pd.get_dummies(train_data['nationality']).values
y_val_nationality = pd.get_dummies(val_data['nationality']).values

y_train_age = train_data['age'].values
y_val_age = val_data['age'].values

y_train_dress_color = pd.get_dummies(train_data['dress_color']).values
y_val_dress_color = pd.get_dummies(val_data['dress_color']).values

y_train_emotion = pd.get_dummies(train_data['emotion']).values
y_val_emotion = pd.get_dummies(val_data['emotion']).values

# Train the model
history = model.fit(X_train, 
                    {'nationality': y_train_nationality, 'age': y_train_age, 'dress_color': y_train_dress_color, 'emotion': y_train_emotion},
                    validation_data=(X_val, {'nationality': y_val_nationality, 'age': y_val_age, 'dress_color': y_val_dress_color, 'emotion': y_val_emotion}),
                    epochs=10,
                    batch_size=32)


ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.